In [1]:
import torch
import torchtext

print(torch.__version__)
print(torchtext.__version__)

C:\Users\user\AppData\Roaming\Python\Python38\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


1.9.1+cpu
0.10.1


In [2]:
import pandas as pd
from konlpy.tag import Okt

# 데이터 불러오기
df = pd.read_csv('comments.txt', names=['comments'])

# Okt 형태소 분석기 객체 생성
okt = Okt()

# 감정 사전 불러오기
knu_dict = pd.read_csv('KNU.csv')
knu_dict = dict(zip(knu_dict['word'], knu_dict['polarity']))

# comments 칼럼을 형태소 분석한 뒤 감정 점수 계산하기
sentiment_scores = []
for comment in df['comments']:
    words = okt.morphs(comment)
    print(words)
    score = sum([knu_dict.get(word, 0) for word in words])
    score = score if score >= 0 else 0
    if score > 0:
        sentiment_scores.append(0) #일상
    else:
        sentiment_scores.append(1) #갈등, 자극적 성향

df['comments']=df.comments.apply(okt.morphs)
# 결과를 라벨링된 데이터프레임으로 변환하기
comments = pd.DataFrame({'comments': df['comments'], 'label': sentiment_scores})

comments

['comments']
['여러분', '은', '지금', '서울대', '공과대학', '박사학위', '취득', '후', '글', '쓰고', '자기', '팬미팅', '자기', '가', '열고', '거기', '서', '의사', '여친', '만나', '결혼', '하고', '아이', '사준', '트램펄린', '위', '에서', '더브', '덤블링', '하다', '척추', '나가고', '회복', '후', '육아', '만화', '그리다가', '암', '걸리고', '완치', '받은', '작가', '님', '을', '보고', '계십니다', '.']
['작가', '님', '암', '4', '기', '판정', '받았다고요', '?', '완치', '?']
['미쳤다', '작가', '님', '오셨다', '작가', '님', '육아', '일기', '할', '때', '박사', '과정', '이었는데', '저', '박사', '학위', '땄습니다', '.', '작가', '님', '이', '왜', '산', '의', '정상', '에', '오르냐의', '이유', '를', '표현', '하신', '적', '있었는데', '할', '수', '있다를', '나', '는', '해냈다로', '바꾸기', '위', '해서라', '하셨죠', '.', '그', '말', '마음속', '에', '깊이', '새기고', '힘낼', '수', '있었습니다', '.', '너무', '괴롭고', '힘들었는데', '해냈습니다', '.']
['작가', '님', '프롤로그', '부터', '울리시기', '있나요', '?', 'ㅜㅜ', '너무', '기다렸어요', '!', '닥액닥', '속편', '같은', '느낌', '으로', '시작', '해서', '더', '좋네요', '!', '이번', '작도', '기대하며', '보겠습니다', '!']
['죽음', '의', '경지', '를', '두', '번', '이나', '극복', '하신', '정대만', '처럼', '포기', '를', '모르는', '닥터', '베르', '님', '의', '화려한', '복귀', '

comments  label
0                                           [comments]      1
1    [여러분, 은, 지금, 서울대, 공과대학, 박사학위, 취득, 후, 글, 쓰고, 자기...      0
2                [작가, 님, 암, 4, 기, 판정, 받았다고요, ?, 완치, ?]      0
3    [미쳤다, 작가, 님, 오셨다, 작가, 님, 육아, 일기, 할, 때, 박사, 과정,...      0
4    [작가, 님, 프롤로그, 부터, 울리시기, 있나요, ?, ㅜㅜ, 너무, 기다렸어요,...      0
..                                                 ...    ...
96   [어른, 되고, 나서, 보, 니까, 그러려니, 하는데, 이번, 에피, 는, 좀, 소...      1
97                   [나도, 고마워, 미래, 가, 웃는, 모습, 보게, 해줘서]      0
98             [희나의, 정체, 를, 알, 고, 보니, 그저, 웃겨, 보임, ㅋㅋㅋ]      0
99          [백합, 이, 가, 미래, 를, 질투, 하는, 게, 딱, 보이, 네, ..]      1
100                               [미래, 마음, 이, 아프네, ..]      1

[101 rows x 2 columns]

In [3]:
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
# Word2Vec 모델 학습
w2v_model = Word2Vec(comments['comments'], 
                     sg=1, 
                     vector_size=100, 
                     window=2, 
                     min_count=1, 
                     workers=4)

# 모델 저장
w2v_model.wv.save_word2vec_format('word2vec')

# 모델 불러오기
w2v_model = KeyedVectors.load_word2vec_format('word2vec', binary=False)

In [4]:
from numpy.random import RandomState

rng=RandomState()

tr=comments.sample(frac=0.8, random_state=rng)
val=comments.loc[~comments.index.isin(tr.index)]

tr.to_csv('train.csv', index=False, encoding='utf-8-sig')
val.to_csv('validation.csv', index=False, encoding='utf-8-sig')

In [5]:
import torchtext
import re
from torchtext.legacy.data import Field, TabularDataset

def tokenizer(text):
    text=re.sub('[\[\]\']','',str(text))
    text=text.split(', ')
    return text

TEXT =Field(tokenize=tokenizer)
LABEL=Field(sequential=False, use_vocab=True)

In [6]:
train, validation = TabularDataset.splits(
    path='.',
    train='train.csv',
    validation='validation.csv',
    format='csv',
    fields=[('text', TEXT), ('label', LABEL)],
    skip_header=True
)

print("Train:",train[0].text, train[0].label)
print("Val:",validation[0].text, validation[0].label)
print(len(train),len(validation))

Train: ['ㅋㅋㅋ', '무슨', '로맨스', '영화', '같이', '연애', '하셨네요'] 0
Val: ['comments'] 1
81 20


In [7]:
import torch
from torchtext.vocab import Vectors
from torchtext.legacy.data import BucketIterator

vectors=Vectors(name='word2vec')
TEXT.build_vocab(train, vectors=vectors, min_freq=1, max_size=None)

LABEL.build_vocab(train)

vocab=TEXT.vocab

device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_itr,validation_itr=BucketIterator.splits(
    datasets=(train, validation),
    batch_size=8,
    device=device,
    sort=False
)

print('임베딩 벡터의 개수와 차원 : {}'.format(TEXT.vocab.vectors.shape))

임베딩 벡터의 개수와 차원 : torch.Size([677, 100])


In [8]:
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

class TextCNN(nn.Module):
    def __init__(self, vocab_built, emb_dim, dim_channel, kernel_wins, num_class):
        super(TextCNN, self).__init__()
        
        self.embed=nn.Embedding(len(vocab_built), emb_dim)
        self.embed.weight.data.copy_(vocab_built.vectors)
        
        self.convs=nn.ModuleList([nn.Conv2d(1, dim_channel, (w, emb_dim)) for w in kernel_wins])
        self.relu=nn.ReLU()
        self.dropout=nn.Dropout(0.5)
        
        self.fc=nn.Linear(len(kernel_wins)*dim_channel, num_class)
        
    def forward(self, x):
        emb_x=self.embed(x)
        emb_x=emb_x.unsqueeze(1)
        
        con_x=[self.relu(conv(emb_x)) for conv in self.convs]
        
        pool_x=[F.max_pool1d(x.squeeze(-1), x.size()[2]) for x in con_x]
        
        fc_x=torch.cat(pool_x, dim=1)
        fc_x=fc_x.squeeze(-1)
        fc_x=self.dropout(fc_x)
        
        logit=self.fc(fc_x)
        
        return logit

In [9]:
def train(model, device, train_itr, optimizer):
    model.train()
    corrects, train_loss=0.0, 0
    
    for batch in train_itr:
        
        text, target=batch.text, batch.label
        text=torch.transpose(text, 0, 1)
        target.data.sub_(1)
        text, target=text.to(device), target.to(device)
        
        optimizer.zero_grad()
        logit=model(text)
        
        loss=F.cross_entropy(logit, target)
        loss.backward()
        optimizer.step()
        
        train_loss+=loss.item()
        result=torch.max(logit, 1)[1]
        corrects+=(result.view(target.size()).data==target.data).sum()
        
    train_loss/=len(train_itr.dataset)
    accuracy=100.0*corrects/len(train_itr.dataset)
    
    return train_loss, accuracy

In [10]:
def evaluate(model, device, itr):
    model.eval()
    corrects, test_loss=0.0, 0
    
    for batch in itr:
        text=batch.text
        target=batch.label
        text=torch.transpose(text, 0, 1)
        target.data.sub_(1)
        text, target=text.to(device), target.to(device)
        
        logit=model(text)
        loss=F.cross_entropy(logit, target)
        
        test_loss+=loss.item()
        result=torch.max(logit, 1)[1]
        corrects+=(result.view(target.size()).data==target.data).sum()
        
    test_loss/=len(itr.dataset)
    accuracy=100.0*corrects/len(itr.dataset)
    
    return test_loss, accuracy

In [11]:
model=TextCNN(vocab, 100, 10, [1, 2, 3], 2).to(device)
print(model)

device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
optimizer=optim.Adam(model.parameters(), lr=0.001)

best_test_acc = -1

for epoch in range(1, 20+1):
    
    tr_loss, tr_acc=train(model, device, train_itr, optimizer)
    print('Train Epoch:{}\tLoss:{}\tAccuracy:{}'.format(epoch, tr_loss, tr_acc))
    
    val_loss, val_acc=evaluate(model, device, validation_itr)
    
    print('Valid Epoch:{}\tLoss:{}\tAccuracy:{}'.format(epoch, val_loss, val_acc))
    
    if val_acc > best_test_acc:
        best_test_acc=val_acc
        
        print("model saves at {} accuracy".format(best_test_acc))
        torch.save(model.state_dict(), "R_TextCNN_Best_Validation")
    
    print('--------------------------------------------------------------------------')

TextCNN(
  (embed): Embedding(677, 100)
  (convs): ModuleList(
    (0): Conv2d(1, 10, kernel_size=(1, 100), stride=(1, 1))
    (1): Conv2d(1, 10, kernel_size=(2, 100), stride=(1, 1))
    (2): Conv2d(1, 10, kernel_size=(3, 100), stride=(1, 1))
  )
  (relu): ReLU()
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=30, out_features=2, bias=True)
)
Train Epoch:1	Loss:0.09292919326711584	Accuracy:56.790122985839844
Valid Epoch:1	Loss:0.09961494505405426	Accuracy:70.0
model saves at 70.0 accuracy
--------------------------------------------------------------------------
Train Epoch:2	Loss:0.09206967294952016	Accuracy:56.790122985839844
Valid Epoch:2	Loss:0.0990676611661911	Accuracy:70.0
--------------------------------------------------------------------------
Train Epoch:3	Loss:0.09272894815162376	Accuracy:56.790122985839844
Valid Epoch:3	Loss:0.09855553209781646	Accuracy:70.0
--------------------------------------------------------------------------
Train Epoch:4	Loss:0

In [14]:
model.load_state_dict(torch.load("R_TextCNN_Best_Validation"))

def preprocess_test_data(text):
    okt = Okt()
    morphs = okt.morphs(text, stem=True)
    tokens = [vocab[token] for token in morphs]
    return tokens

#test_data = preprocess_test_data("추한모랑 불륜 하면서 우리 윤식이 옆에 하트 붙인 쟤는 양심이 없나? 지 딴에 죄책감이라는 게 또 있나봄 태린이 보니까 피하는 거 봐라 ㅋㅋ")
#test_data = preprocess_test_data("답답하고 짜증나기는 처음이다. 답답한 남자와 자기 세상 밖에 모르는 이기적인 여자의 얘기네")
#test_data = preprocess_test_data("주인공이 참 예쁘네. 말꼬라지가")
#test_data=preprocess_test_data("주인공 참 예쁜데 머리가 꽃밭인가봐")
#test_data = preprocess_test_data("주인공 넘 행복해 보인다 정말 다행이야 작가님 화이팅!")
test_data = preprocess_test_data("ㅋㅋㅋ 진짜 남주 넘넘 귀여워 강아지같아")
#test_data=preprocess_test_data("이 웹툰 힐링 웹툰임 ㅋㅋㅋ 사랑스러워 등장인물들 다")
#test_data=preprocess_test_data("개웃겨 ㅋㅋㅋ 진짜 어이 없는데 웃기다")

test_tensor = torch.LongTensor(test_data).unsqueeze(dim=0).to(device)

model.eval()
with torch.no_grad():
    output = model(test_tensor)
    pred = output.argmax(dim=-1).item()

if pred == 1:
    print("test data : 일반 맥락 인지")
else:
    print("test data : 갈등 맥락 인지")

test data : 일반 맥락 인지
